# Data Loading

In [ ]:
#get tabular data
import pandas as pd
data_tabular= pd.read_csv("/content/drive/MyDrive/Datathon 2025 shared space/Data/merged_training_data_final.csv")
print(data_tabular.shape)
print(data_tabular.columns)


In [ ]:
#drop global features
#data_tabular.drop(columns=["subject", "wiener", "avg_path", "stoer_wagner_cut", "dijkstra_path_len", "matching_size", "barycenter_count", "cut_size", "conductance", "random_cut"], inplace=True)
#drop non-significant features
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("MRI_Track_Scan_")], inplace=True)
data_tabular.drop(columns=["ADHD_Outcome_1", "ADHD_Outcome_0", "Sex_F_0", "Sex_F_1"], inplace=True )
#data_tabular.drop(columns=["MRI_Track_Age_at_Scan", "APQ_P_APQ_P_CP", "EHQ_EHQ_Total", "APQ_P_APQ_P_PM", "APQ_P_APQ_P_PP", "ColorVision_CV_Score"], inplace=True )
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Basic_Demos_Enroll_Year_")], inplace=True)
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Basic_Demos_Study_Site")], inplace=True)
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Barratt_Barratt_P1_Occ")], inplace=True)
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Barratt_Barratt_P2_Occ")], inplace=True)
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Barratt_Barratt_P1_Edu")], inplace=True)
data_tabular.drop(columns=[col for col in data_tabular.columns if col.startswith("Barratt_Barratt_P2_Edu")], inplace=True)

In [ ]:
print(data_tabular.shape)
data_tabular.columns

In [ ]:
#Three methods of local features
#path1= /content/drive/MyDrive/Datathon 2025 shared space/Data/Local features/method3_selected_four_features.csv
#path2= /content/drive/MyDrive/Datathon 2025 shared space/Data/Local features/top20_from_wrapper_stattest_method2_local_features.csv
#path3 = /content/drive/MyDrive/Datathon 2025 shared space/Data/Local features/overlapping_local_features_wrapper_stattest.csv

data_local=pd.read_csv('/content/drive/MyDrive/Datathon 2025 shared space/Data/Local features/overlapping_local_features_wrapper_stattest.csv')


In [ ]:
data_merged=data_tabular.merge(data_local, on="participant_id")

In [ ]:
print(data_merged.shape)

(1212, 36)


# XGBOOST

In [ ]:
# Encode combined target: 2 binary labels → 4 classes (0 to 3)
y = data_merged[["ADHD_Outcome", "Sex_F"]].apply(lambda row: int(row["ADHD_Outcome"]) * 2 + int(row["Sex_F"]), axis=1)
X = data_merged.drop(columns=["participant_id", "ADHD_Outcome", "Sex_F"])
print(y)


0       2
1       2
2       1
3       1
4       2
       ..
1207    1
1208    2
1209    3
1210    2
1211    1
Length: 1212, dtype: int64


In [ ]:
print(X.columns)

Index(['APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
       'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
       'PreInt_Demos_Fam_Child_Ethnicity_0.0',
       'PreInt_Demos_Fam_Child_Ethnicity_1.0',
       'PreInt_Demos_Fam_Child_Race_0.0', 'PreInt_Demos_Fam_Child_Race_1.0',
       'PreInt_Demos_Fam_Child_Race_2.0', 'PreInt_Demos_Fam_Child_Race_3.0',
       'PreInt_Demos_Fam_Child_Race_4.0', 'PreInt_Demos_Fam_Child_Race_7.0',
       'PreInt_Demos_Fam_Child_Race_8.0', 'PreInt_Demos_Fam_Child_Race_9.0',
       'Current Flow Betweenness_ROI100', 'Eigenvector Centrality_ROI4',
       'Eigenvector Centrality_ROI45', 'PageRank_ROI189', 'PageRank_ROI4',
       'Degree_ROI126', 'Information Centrality_ROI10',
       'Information Centrality_ROI174', 'Information Centrality_R

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
import numpy as np

# Assume X and y are already defined (X: features, y: target labels)

# Split the data (stratified to preserve class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.1,
    stratify=y,
    random_state=42
)

# Compute class weights and sample weights for training data
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))
sample_weights = np.array([class_weights_dict[label] for label in y_train])

# Instantiate and train the model
model = XGBClassifier(
    objective="multi:softprob",
    num_class=4,
    learning_rate=0.05,
    max_depth=7,
    n_estimators=300,
    eval_metric='mlogloss',
    random_state=42
)

model.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluate on test set
y_pred = model.predict(X_test)
print("Test Set Evaluation:\n")
print(classification_report(y_test, y_pred, digits=3))
